# Testando Kalman-Filter

In [377]:
import cv2
import numpy as np
import pandas as pd
import itertools

## Funções

In [378]:
def detectHarrisKeypoints(image, threshold=0.01, blockSize=2, ksize=3, k=0.04):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    gray_image_f32 = np.float32(gray_image)

    # Applying the function 
    dst = cv2.cornerHarris(gray_image_f32, blockSize, ksize, k) 
  
    # dilate to mark the corners 
    dst = cv2.dilate(dst, None)
    
    ret, dst = cv2.threshold(dst,threshold*dst.max(),255,0)
    dst = np.uint8(dst)

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

    # define the criteria to stop and refine the corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(gray_image_f32,np.float32(centroids),(5,5),(-1,-1),criteria)

    # # extract keypoints
    # points = np.argwhere(dst > threshold * dst.max())
    
    keypoints = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in corners]

    # draw keypoints
    # image[dst > threshold * dst.max()] = [0, 255, 0]
    kp_image = cv2.drawKeypoints(image, keypoints, None, color=(255, 0, 0), flags=0)

    return keypoints, kp_image

In [379]:
def detectSIFTKeypoints(image, nfeatures=0, nOctaveLayers=3, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6, enable_precise_upscale=False):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
    # Applying the function 
    sift = cv2.SIFT_create(nfeatures, nOctaveLayers, contrastThreshold, edgeThreshold, sigma, enable_precise_upscale) 
    kp, des = sift.detectAndCompute(gray_image, None)
    not_dup_kp = {pt.pt: pt for pt in kp}
    kp_ = list(not_dup_kp.values())
    # Applying the function 
    kp_image = cv2.drawKeypoints(image, kp_, None, color=(0, 255, 0), flags=0)

    return kp_, kp_image 

def detectSIFTKeypointsFilter(image, nfeatures=0, nOctaveLayers=3, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6, enable_precise_upscale=False):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    gray_image_f32 = np.float32(gray_image)
  
    # Applying the function 
    sift = cv2.SIFT_create(nfeatures, nOctaveLayers, contrastThreshold, edgeThreshold, sigma, enable_precise_upscale) 
    kp, des = sift.detectAndCompute(gray_image, None)
    not_dup_kp = {pt.pt: pt for pt in kp}
    kp_ = list(not_dup_kp.values())
    
    altura, largura = image.shape[:2]
    imagem_binaria = np.zeros((altura, largura), dtype=np.uint8)
    for kp in kp_:
        x, y = map(int, kp.pt)
        cv2.circle(imagem_binaria, (x, y), 5, 255, -1)

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(imagem_binaria)

    # define the criteria to stop and refine the corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(gray_image_f32,np.float32(centroids),(5,5),(-1,-1),criteria)

    # # extract keypoints
    # points = np.argwhere(dst > threshold * dst.max())
    
    keypoints = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in corners]
    
    # Applying the function 
    kp_image = cv2.drawKeypoints(image, keypoints, None, color=(0, 255, 0), flags=0)

    return keypoints, kp_image 

In [380]:
def detectStarKeypoints(image, max_size = 41, response_threshold = 30, line_threshold_projected = 10,
                        line_threshold_binarized = 8, suppress_nonmax_size = 5):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
    # Applying the function 
    star = cv2.xfeatures2d.StarDetector_create(maxSize= max_size, 
                                        responseThreshold = response_threshold,
                                        lineThresholdProjected = line_threshold_projected,
                                        lineThresholdBinarized = line_threshold_binarized,
                                        suppressNonmaxSize = suppress_nonmax_size)
    kp = star.detect(gray_image, None)

    # Applying the function 
    kp_image = cv2.drawKeypoints(image, kp, None, color=(0, 0, 255), flags=0) 

    return kp, kp_image 


def detectStarKeypointsFilter(image, max_size = 41, response_threshold = 30, line_threshold_projected = 10,
                        line_threshold_binarized = 8, suppress_nonmax_size = 5):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_image_f32 = np.float32(gray_image)
  
    # Applying the function 
    star = cv2.xfeatures2d.StarDetector_create(maxSize= max_size, 
                                        responseThreshold = response_threshold,
                                        lineThresholdProjected = line_threshold_projected,
                                        lineThresholdBinarized = line_threshold_binarized,
                                        suppressNonmaxSize = suppress_nonmax_size)
    kps = star.detect(gray_image, None)

    altura, largura = image.shape[:2]
    imagem_binaria = np.zeros((altura, largura), dtype=np.uint8)
    for kp in kps:
        x, y = map(int, kp.pt)
        cv2.circle(imagem_binaria, (x, y), 5, 255, -1)

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(imagem_binaria)

    # define the criteria to stop and refine the corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(gray_image_f32,np.float32(centroids),(5,5),(-1,-1),criteria)

    # # extract keypoints
    # points = np.argwhere(dst > threshold * dst.max())
    
    keypoints = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in corners]

    # Applying the function 
    kp_image = cv2.drawKeypoints(image, keypoints, None, color=(0, 0, 255), flags=0) 

    return keypoints, kp_image 

In [381]:
from sklearn.metrics import pairwise_distances_argmin_min

def computeDistacesKeypoints(pts1, pts2, threshold=1):
    array_pts1 = np.asarray(pts1)
    array_pts2 = np.asarray(pts2)

    if array_pts1.shape[0] > 0 and array_pts2.shape[0] > 0:
        dists = pairwise_distances_argmin_min(array_pts1, array_pts2)
        matches_pts1 = [pts1[i] for i, (pt, dist) in enumerate(zip(dists[0], dists[1])) if dist <= threshold]
        matches_pts2 = [pts2[pt] for pt, dist in zip(dists[0], dists[1]) if dist <= threshold]
    else:
        matches_pts1 = []
        matches_pts2 = []
        
    return matches_pts1, matches_pts2


def return_detector_func(algol):
    if algol == 'harris':
        func = detectHarrisKeypoints
    elif algol == 'sift_filter':
        func = detectSIFTKeypointsFilter
    elif algol == 'sift':
        func = detectSIFTKeypoints
    elif algol == 'star_filter':
        func = detectStarKeypointsFilter
    else:
        func = detectStarKeypoints
    
    return func


def compare_detectors_keypoints(image, algol_1, algol_2, params_algol_1, params_algol_2, threshold):
    detector_1 = return_detector_func(algol_1)
    detector_2 = return_detector_func(algol_2)

    kp1, _ = detector_1(image, **params_algol_1)
    kp2, _ = detector_2(image, **params_algol_2)

    kp1 = [(kp.pt[0], kp.pt[1]) for kp in kp1]
    kp2 = [(kp.pt[0], kp.pt[1]) for kp in kp2]

    matches_pts1, matches_pts2 = computeDistacesKeypoints(kp1, kp2, threshold=threshold)
    mismatches_pts1 = [pt for pt in kp1 if pt not in matches_pts1]
    mismatches_pts2 = [pt for pt in kp2 if pt not in matches_pts2]

    matches_kp1 = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in matches_pts1]
    matches_kp2 = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in matches_pts2]

    mismatches_kp1 = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in mismatches_pts1]
    mismatches_kp2 = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in mismatches_pts2]

    kp_image = cv2.drawKeypoints(image, matches_kp1, None, color=(0, 255, 0), flags=0)
    kp_image = cv2.drawKeypoints(kp_image, matches_kp2, None, color=(0, 255, 0), flags=0)
    kp_image = cv2.drawKeypoints(kp_image, mismatches_kp1, None, color=(255, 0, 0), flags=0)
    kp_image = cv2.drawKeypoints(kp_image, mismatches_kp2, None, color=(0, 0, 255), flags=0)

    return kp_image, matches_kp1, matches_kp2, kp1, kp2 

def return_params_combination(combination, params_harris, params_sift, params_star, pos):
    if combination == 'harris':
        params = {key: value[pos] for key, value in zip(params_harris.keys(), params_harris.values())}
    elif combination == 'sift' or combination == 'sift_filter':
        params = {key: value[pos] for key, value in zip(params_sift.keys(), params_sift.values())}
    else:
        params = {key: value[pos] for key, value in zip(params_star.keys(), params_star.values())}
    
    return params


def run_all_tests(image, image_name, params_harris, params_sift, params_star, thresholds, filter=False, n_pontos=['25', '50', '100']):
    if filter:
        algol_combinations = [('harris', 'sift_filter'), ('harris', 'star_filter'), ('sift_filter', 'star_filter')]
    else:    
        algol_combinations = [('harris', 'sift'), ('harris', 'star'), ('sift', 'star')]

    df = pd.DataFrame(columns=['combination', 'qtd_kp1', 'qtd_kp2', 'qtd_matches_kp1',
                                'qtd_matches_kp2', 'precision_1', 'precision_2'])
    
    for comb in algol_combinations:
        for i,n in enumerate(n_pontos):
            image_ = image.copy()
            params1 = return_params_combination(comb[0], params_harris, params_sift, params_star, i)
            params2 = return_params_combination(comb[1], params_harris, params_sift, params_star, i)
            
            result = compare_detectors_keypoints(image_, comb[0], comb[1], params1, params2, thresholds[i])
            
            kp_image, matches_kp1, matches_kp2, kp1, kp2 = result


            df.loc[len(df.index)] = [comb, len(kp1), len(kp2), len(matches_kp1), len(matches_kp2), len(matches_kp1) / len(kp1),
                               len(matches_kp2) / len(kp2) ]

            path_img = f'../results/first_part_detectors/{image_name}_matches_{comb[0]}_{comb[1]}_{n}_pontos.jpg'
            cv2.imwrite(path_img, kp_image)

    return df

In [382]:
import copy

def draw_matches(train_img, query_img, trainKeypoints, queryKeypoints, matches, matchesMask):
    kps_train = [[trainKeypoints[matches[i][0].trainIdx].pt[0], trainKeypoints[matches[i][0].trainIdx].pt[1]] for i, m in enumerate(matchesMask)]
    kps_query = [[queryKeypoints[matches[i][0].queryIdx].pt[0], queryKeypoints[matches[i][0].queryIdx].pt[1]] for i, m in enumerate(matchesMask)]

    # draw the tracks
    mask = np.zeros_like(train_img, 'uint8') 

    line = (0,255,0)
    point_true = (255,0,0)
    point_false = (0,0,255)
    frame = query_img.copy()
    
    for i, (m, query, train) in enumerate(zip(matchesMask, kps_query, kps_train)):
        a, b = query[0], query[1]
        c, d = train[0], train[1]
        
        a, b = int(a), int(b)
        c, d = int(c), int(d)
        
        if m[0] > 0:
            mask = cv2.line(mask, (a, b), (c, d), line, 2) 
            frame = cv2.circle(frame, (a, b), 4, point_true, -1)
        else:
            frame = cv2.circle(frame, (a, b), 4, point_false, -1)
    
    img = cv2.add(frame, mask)

    return img


from sklearn.metrics import pairwise_distances_argmin_min

def return_idx_points_at_distance(pts1, pts2, threshold=1):
    array_pts1 = np.asarray(pts1)
    array_pts2 = np.asarray(pts2)

    if array_pts1.shape[0] > 0 and array_pts2.shape[0] > 0:
        dists = pairwise_distances_argmin_min(array_pts1, array_pts2)
        matches = [(i, pos) for i, (pos, dist) in enumerate(zip(dists[0], dists[1])) if dist <= threshold]
    else:
        matches = []
        
    return matches


def filter_matches(kpts1, kpts2, matches, threshold=1):
    kp1 = [(kp.pt[0], kp.pt[1]) for kp in kpts1]
    kp2 = [(kp.pt[0], kp.pt[1]) for kp in kpts2]
    
    matches_distance_idx = return_idx_points_at_distance(kp1, kp2, threshold)
    matchesMask_ = [ [1,0] if (m[0].trainIdx, m[0].queryIdx) in matches_distance_idx else [0,0] for m in matches]

    return matchesMask_


def compare_descriptors(keypoints1, keypoints2, descriptors1,  descriptors2, window_size, threshold):

    # Iterate through descriptors in the second image and compare with descriptors in the window
    matches = []
    for i, descriptor2 in enumerate(descriptors2):
        x, y = keypoints2[i].pt
        distances = []

        # Define the window coordinates
        window_x1 = int(x - window_size / 2)
        window_y1 = int(y - window_size / 2)
        window_x2 = int(x + window_size / 2)
        window_y2 = int(y + window_size / 2)

        # Ensure the window is within image bounds
        for j, descriptor1 in enumerate(descriptors1):
            x2, y2 = keypoints1[j].pt
            if x2 > window_x1 and y2 > window_y1  and \
                x2 <= window_x2 and y2 <= window_y2 :

                # Calculate Euclidean distance between descriptor1 and descriptors in the window
                distance = np.linalg.norm(descriptor2 - descriptor1)
                distances.append((j, distance))

        # Check if any descriptor in the window is below the threshold
        distances = sorted(distances, key=lambda x: x[1])
        if len(distances) > 0 and distances[0][1] <= threshold:
            duplicate = [m for m in matches if distances[0][0] == m[0].queryIdx]
            if len(duplicate) > 0: continue
            
            match = cv2.DMatch(_queryIdx=distances[0][0], _trainIdx=i, _distance=np.min(distances))
            matches.append((match, match))
    
    matches = tuple(tuple(m) for m in matches)
    return matches


def matchDetectedKeypoints(train_img, query_img, threshold=5, window_size=30, detector='harris', descriptor='sift', dect_kargs={}, des_kargs={}):
    query_img_bw = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY) 
    train_img_bw = cv2.cvtColor(train_img, cv2.COLOR_BGR2GRAY)

    if descriptor == 'brief':
        des = cv2.xfeatures2d.BriefDescriptorExtractor_create(**des_kargs)
        norm_type = cv2.NORM_HAMMING
    elif descriptor == 'brisk':
        des = cv2.BRISK_create(**des_kargs)
        norm_type = cv2.NORM_HAMMING
    else:
        des = cv2.SIFT.create(**des_kargs)
        norm_type = cv2.NORM_L2
    
    if detector == 'harris':
        func_var = detectHarrisKeypoints
    elif detector == 'sift_filter':
        func_var = detectSIFTKeypointsFilter
    elif detector == 'sift':
        func_var = detectSIFTKeypoints
    elif detector == 'star_filter':
        func_var = detectStarKeypointsFilter
    else:
        func_var = detectStarKeypoints

    queryKeypoints, _ = func_var(query_img, **dect_kargs)
    trainKeypoints, _ = func_var(train_img, **dect_kargs)
    
    _,queryDescriptors = des.compute(query_img_bw, queryKeypoints) 
    _,trainDescriptors = des.compute(train_img_bw, trainKeypoints)

    print(len(trainKeypoints))
    # matcher = cv2.BFMatcher(normType=norm_type, crossCheck=False) 
    # matches = matcher.knnMatch(queryDescriptors, trainDescriptors, k=2)

    matches = compare_descriptors(queryKeypoints, trainKeypoints, 
                                  queryDescriptors, trainDescriptors, window_size, threshold)
    
    # Need to draw only good matches, so create a mask
    matchesMask = [[1,0] for i in range(len(matches))]

    # for i,(m,n) in enumerate(matches):
    #     if m.distance < ratio_test*n.distance:
    #         matchesMask[i]=[1,0]

    # matchesMask = filter_matches(trainKeypoints, queryKeypoints, matches, threshold=threshold)

    draw_params = dict(
                    matchColor = (0,255,0),
                    singlePointColor = (255,0,0),
                    matchesMask = matchesMask,
                    flags = cv2.DrawMatchesFlags_DEFAULT)
    
    final_img = cv2.drawMatchesKnn(query_img, queryKeypoints, train_img, trainKeypoints, matches, None,**draw_params)
    
    qtd_kp_train = len(trainKeypoints)
    qtd_kp_query = len(queryKeypoints)
    qtd_matches = len(matches)

    final_img = cv2.resize(final_img, (1280,480))

    return final_img, qtd_kp_train, qtd_kp_query, qtd_matches


def return_params_combination_descriptors(combination, params_harris, params_sift, params_star, params_brisk, pos):
    if combination == 'harris':
        params = {key: value[pos] for key, value in params_harris.items()}
    elif combination == 'sift' or combination == 'sift_filter':
        params = {key: value[pos] for key, value in params_sift.items()}
    elif combination == 'star' or combination == 'star_filter':
        params = {key: value[pos] for key, value in params_star.items()}
    else:
        params = {key: value[pos] for key, value in params_brisk.items()}
    return params


def run_all_tests_descriptors(train_img, query_img, image_name, params_harris, params_sift, params_star, params_brisk, thresholds={'sift': 100, 'brisk': 1000},
                              window_size=50, pos=1, filter=False, n_pontos=['25', '50', '100']):
    
    if filter:
        algol_combinations = [('harris', 'sift'), ('harris', 'brisk'), ('star_filter', 'sift'), ('star_filter', 'brisk'), ('sift_filter', 'sift'), ('sift_filter', 'brisk')]
    else:
        algol_combinations = [('harris', 'sift'), ('harris', 'brisk'), ('star', 'sift'), ('star', 'brisk'), ('sift', 'sift'), ('sift', 'brisk')]
    df = pd.DataFrame(columns=['detector', 'descriptor', 'qtd_kp1', 'qtd_kp2', 'qtd_matches', 'precision_1', 'precision_2'])
    
    for comb in algol_combinations:
        params_det = return_params_combination_descriptors(comb[0], params_harris, params_sift, params_star, params_brisk, pos)
        params_des = return_params_combination_descriptors(comb[1], params_harris, params_sift, params_star, params_brisk, pos)
        
        result = matchDetectedKeypoints(train_img, query_img, thresholds[comb[1]], window_size, detector=comb[0], descriptor=comb[1],
                                         dect_kargs=params_det, des_kargs=params_des)
        
        matches_image, qtd_kp_train, qtd_kp_query, qtd_matches = result


        df.loc[len(df.index)] = [comb[0], comb[1], qtd_kp_train, qtd_kp_query, qtd_matches, qtd_matches / qtd_kp_train,
                            qtd_matches / qtd_kp_query ]

        path_img = f'../results/second_part_descriptors/{image_name}_matches_{comb[0]}_{comb[1]}_{n_pontos[pos]}_pontos.jpg'
        cv2.imwrite(path_img, matches_image)

    return df


In [724]:
def return_points_hs(img, flow, x, y):
    # h, w = img.shape[:2]
    # y, x = np.mgrid[step / 2:h:step, step / 2:w:step].reshape(2, -1).astype(int)

    fx, fy = flow[y, x].T
    lines = np.vstack([x, y, x + fx, y + fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines)

    old_points = np.asarray([[x1, y1] for (x1, y1), (x2, y2) in lines])
    new_points = np.asarray([[x2, y2] for (x1, y1), (x2, y2) in lines])

    return old_points, new_points

def horn_schunck_opflow(train_img, query_img, func_detector, feature_params, params_horn_schunck):
    # Convert to gray scale
    prvs = cv2.cvtColor(train_img, cv2.COLOR_BGR2GRAY)
    
    # Capture another frame and convert to gray scale
    next = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY)

    # Optical flow is now calculated
    flow = cv2.calcOpticalFlowFarneback(prev=prvs, next=next,flow=None, flags=0, **params_horn_schunck)

    # Calculated fp points
    kps,_ = func_detector(train_img, **feature_params)
    x = np.array([int(kp.pt[0]) for kp in kps])
    y = np.array([int(kp.pt[1]) for kp in kps])
    old_points, new_points = return_points_hs(next, flow, x, y)

    old_points = np.array([[[int(kp[0]), int(kp[1])]] for kp in old_points])
    new_points = np.array([[[int(kp[0]), int(kp[1])]] for kp in new_points])

    return old_points, new_points


def horn_schunck_opflow_KF(train_img, query_img, points, params_horn_schunck):
    # Convert to gray scale
    prvs = cv2.cvtColor(train_img, cv2.COLOR_BGR2GRAY)
    
    # Capture another frame and convert to gray scale
    next = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY)

    # Optical flow is now calculated
    flow = cv2.calcOpticalFlowFarneback(prev=prvs, next=next,flow=None, flags=0, **params_horn_schunck)

    h, w = next.shape[:2]

    # Calculated fp points
    x = np.array([int(kp[0][0]) if int(kp[0][0]) < w-1 else w-1 for kp in points])
    y = np.array([int(kp[0][1]) if int(kp[0][1]) < h-1 else h-1 for kp in points])
    old_points, new_points = return_points_hs(next, flow, x, y)
    
    old_points = np.array([[[int(kp[0]), int(kp[1])]] for kp in old_points])
    new_points = np.array([[[int(kp[0]), int(kp[1])]] for kp in new_points])

    return old_points, new_points

In [725]:
def lucas_kanade_opflow(train_img, query_img, func_detector, feature_params, lk_params):
    
    # Take first frame and find corners in it 
    old_frame = train_img.copy() 
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    # p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
    kps,_ = func_detector(old_frame, **feature_params)
    p0 = np.array([[[int(kp.pt[0]), int(kp.pt[1])]] for kp in kps])
    p0 = p0.astype(np.float32)

    # Create a mask image for drawing purposes 

    mask = np.zeros_like(old_frame, 'uint8') 
    
    frame = query_img.copy()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 

    # calculate optical flow 
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params) 

    # Select good points 
    good_new = p1
    good_old = p0

    return good_old, good_new


def lucas_kanade_opflow_KF(train_img, query_img, points, lk_params):
    
    # Take first frame and find corners in it 
    old_frame = train_img.copy()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    p0 = np.array([[[int(kp[0][0]), int(kp[0][1])]] for kp in points])
    p0 = p0.astype(np.float32)

    # Create a mask image for drawing purposes 

    mask = np.zeros_like(old_frame, 'uint8') 
    
    frame = query_img.copy()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 

    # calculate optical flow 
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params) 

    # Select good points 
    good_new = p1
    good_old = p0

    return good_old, good_new

## Kalman-filter

In [726]:
class KalmanFilter:

    def __init__(self, coordX, coordY, coordX_, coordY_):
        self.kf = cv2.KalmanFilter(4, 2)
        self.kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
        self.kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
        self.kf.processNoiseCov = np.array(
                [[1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]], np.float32) * 10
        self.kf.statePre = np.array([[coordX], [coordY], [0], [0]], np.float32)
        self.kf.statePost = np.array([[coordX_], [coordY_], [0], [0]], np.float32)


    def predict(self, coordX, coordY):
        ''' This function estimates the position of the object'''
        measured = np.array([[coordX], [coordY]], np.float32)
        self.kf.correct(measured)
        predicted = self.kf.predict()
        x, y = int(predicted[0]), int(predicted[1])
        return x, y

In [727]:
def apply_kalman_filter(image_folders, func_var, dect_kargs, params_opflow, opflow_algol='horn_shunck', detect_algol='harris', n_pontos='50'):
    images = [cv2.resize(cv2.imread(image_folder), (640,480)) for image_folder in image_folders]
    images_names = [ image_name.split('/')[-1].replace('.jpg', '') for image_name in image_folders]

    # p0, p1 = lucas_kanade_opflow(images[0], images[1], func_var, dect_kargs, lk_params)
    p0, p1 = horn_schunck_opflow(images[0], images[1], func_var, dect_kargs, params_opflow)

    # Inicialização do Kalman para cada ponto
    kalman_filters = [KalmanFilter(p1[0][0], p1[0][1], p2[0][0], p2[0][1]) for p1, p2 in zip(p0, p1)]
    kf_points = []
    kf_prev = []

    for i, p in enumerate(p0):
        kf_points.append((p[0][0], p[0][1]))
        kf_prev.append((p[0][0], p[0][1]))

    line_flow = (0,0,255)
    point_flow = (0,0,255)
    line_kf = (0,255,0)
    point_kf = (0,255,0)

    image_name = ''
    # Atualizar cada filtro de Kalman com as novas medições
    for i in range(1, len(images)):
        image_train = images[i-1].copy()
        image_query = images[i].copy()
        
        image_name = images_names[i-1]

        # p0, p1 = lucas_kanade_opflow_KF(image_train, image_query, p0, lk_params)
        p0, p1 = horn_schunck_opflow_KF(image_train, image_query, p0, params_opflow)
        
        for j, (pt_1, pt_2) in enumerate(zip(p0, p1)):
            x, y = pt_1[0][0], pt_1[0][1]
            kf_points[j] = kalman_filters[j].predict(x, y)
            
            x_prev, y_prev = kf_prev[j]
            x_pos, y_pos = kf_points[j]

            # Desenhar a posição estimada
            cv2.circle(image_train, (int(x_prev), int(y_prev)), 5, point_kf, -1)
            cv2.line(image_train, (int(x_prev), int(y_prev)), (int(x_pos), int(y_pos)), line_kf, 2)

            # Desenhar a posição real do detector de Harris
            cv2.circle(image_train, (int(pt_1[0][0]), int(pt_1[0][1])), 5, point_flow, -1)
            cv2.line(image_train, (int(pt_1[0][0]), int(pt_1[0][1])), (int(pt_2[0][0]), int(pt_2[0][1])), line_flow, 2)

            kf_prev[j] = kf_points[j]

        p0 = p1
        path_img = f'../results/fourth_part_kalman_filter/'
        path_img += f'kf_{image_name}_detected_{detect_algol}_{opflow_algol}_{n_pontos}_pontos.jpg'
        cv2.imwrite(path_img, image_train)
        image_name = images_names[i]

    for j, pt_2 in enumerate(p1):
        x, y = pt_2[0][0], pt_2[0][1]
        kf_points[j] = kalman_filters[j].predict(x, y)
        
        x_prev, y_prev = kf_prev[j]
        x_pos, y_pos = kf_points[j]

        # Desenhar a posição estimada
        cv2.circle(image_query, (x_prev, y_prev), 5, point_kf, -1)
        cv2.line(image_query, (x_prev, y_prev), (x_pos, y_pos), line_kf, 2)

        # Desenhar a posição real detector
        cv2.circle(image_query, (int(pt_2[0][0]), int(pt_2[0][1])), 5, (0, 0, 255), -1)

        kf_prev[j] = kf_points[j]

    path_img = f'../results/fourth_part_kalman_filter/'
    path_img += f'kf_{image_name}_detected_{detect_algol}_{opflow_algol}_{n_pontos}_pontos.jpg'
    cv2.imwrite(path_img, image_query)

### dsc07631.jpg

In [728]:
# ## Params img teste 1

lk_params = dict( winSize = (50, 50), 
                  maxLevel = 2, 
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 
                              10, 0.03)) 

params_horn_schunck = dict(pyr_scale=0.5, levels=3, winsize=15,
                           iterations=3, poly_n=5, poly_sigma=1.2)

params_harris = {
    'threshold' : [0.27152, 0.078, 0.053],
    'blockSize' : [2, 5, 3],
    'ksize' : [3, 5, 9],
    'k' : [0.04, 0.04, 0.06]
}

params_sift = {
    'nfeatures': [34, 69, 143],
    'nOctaveLayers' : [3, 10, 10],
    'contrastThreshold' : [0.15, 0.05, 0.01],
    'edgeThreshold' : [2, 10, 3],
    'sigma' : [1.6, 3.2, 3.2],
    'enable_precise_upscale': [True, True, True]
}

params_star = {
    'max_size': [10, 16, 10],
    'response_threshold': [70, 55, 30],
    'line_threshold_projected': [10, 5, 5],
    'line_threshold_binarized': [10, 5, 5],
    'suppress_nonmax_size': [10, 16, 10]
}

dect_kargs = {k: v[1] for k, v in params_harris.items()}
func_var = detectHarrisKeypoints

In [729]:
image_folders = ['../data/imgs/dsc07631.jpg',
                  '../data/imgs/dsc07632.jpg',
                  '../data/imgs/dsc07633.jpg',
                  '../data/imgs/dsc07634.jpg',
                  '../data/imgs/dsc07635.jpg',
        ]

In [730]:
apply_kalman_filter(image_folders, func_var, dect_kargs, params_horn_schunck, opflow_algol='horn_shunck', detect_algol='harris', n_pontos='50')

C:\Users\kdemo\AppData\Local\Temp\ipykernel_9040\518723252.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x, y = int(predicted[0]), int(predicted[1])


### dsc02651.jpg

In [731]:
## Params img teste 2

params_horn_schunck = dict(pyr_scale=0.5, levels=3, winsize=5,
                           iterations=3, poly_n=5, poly_sigma=1.2)

params_harris = {
    'threshold' : [0.43, 0.33, 0.14],
    'blockSize' : [3, 6, 6],
    'ksize' : [3, 5, 5],
    'k' : [0.04, 0.04, 0.04]
}

params_sift = {
    'nfeatures': [37, 60, 125],
    'nOctaveLayers' : [3, 10, 10],
    'contrastThreshold' : [0.15, 0.05, 0.01],
    'edgeThreshold' : [2, 10, 3],
    'sigma' : [1.6, 3.2, 3.2],
    'enable_precise_upscale': [True, True, True]
}

params_star = {
    'max_size': [11, 16, 16],
    'response_threshold': [71, 64, 48],
    'line_threshold_projected': [10, 6, 5],
    'line_threshold_binarized': [5, 6, 5],
    'suppress_nonmax_size': [8, 16, 8]
}

dect_kargs = {k: v[1] for k, v in params_harris.items()}
func_var = detectHarrisKeypoints

In [732]:
image_folders = ['../data/imgs/dsc02651.jpg',
                  '../data/imgs/dsc02652.jpg',
                  '../data/imgs/dsc02653.jpg',
                  '../data/imgs/dsc02654.jpg',
                  '../data/imgs/dsc02655.jpg',
        ]

In [733]:
apply_kalman_filter(image_folders, func_var, dect_kargs, params_horn_schunck, opflow_algol='horn_shunck', detect_algol='harris', n_pontos='50')

C:\Users\kdemo\AppData\Local\Temp\ipykernel_9040\518723252.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x, y = int(predicted[0]), int(predicted[1])


### IMG-20240114-WA0043.jpg

In [734]:
 ## My image
lk_params = dict( winSize = (24, 24), 
                  maxLevel = 1, 
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 
                              10, 0.03)) 


params_horn_schunck = dict(pyr_scale=0.5, levels=3, winsize=10,
                           iterations=3, poly_n=5, poly_sigma=1.2)

params_harris = {
    'threshold' : [0.005],
    'blockSize' : [6],
    'ksize' : [3],
    'k' : [0.04]
}

params_sift = {
    'nfeatures': [55],
    'nOctaveLayers' : [1],
    'contrastThreshold' : [0.02],
    'edgeThreshold' : [5],
    'sigma' : [1.6],
    'enable_precise_upscale': [True]
}

params_star = {
    'max_size': [10],
    'response_threshold': [10],
    'line_threshold_projected': [5],
    'line_threshold_binarized': [5],
    'suppress_nonmax_size': [1]
}

dect_kargs = {k: v[0] for k, v in params_star.items()}
func_var = detectStarKeypointsFilter

In [735]:
image_folders = ['../data/imgs_teste_3/IMG-20240114-WA0043.jpg',
                  '../data/imgs_teste_3/IMG-20240114-WA0044.jpg',
                  '../data/imgs_teste_3/IMG-20240114-WA0045.jpg',
                  '../data/imgs_teste_3/IMG-20240114-WA0046.jpg',
                  '../data/imgs_teste_3/IMG-20240114-WA0047.jpg',
                  '../data/imgs_teste_3/IMG-20240114-WA0048.jpg',
        ]

In [736]:
apply_kalman_filter(image_folders, func_var, dect_kargs, params_horn_schunck, opflow_algol='horn_shunck', detect_algol='star', n_pontos='20')

C:\Users\kdemo\AppData\Local\Temp\ipykernel_9040\518723252.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x, y = int(predicted[0]), int(predicted[1])
